In [1]:
import numpy as np
import pandas as pd
import glob
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

## Currently, trying to figure out a way to introduce variance into the dataset. As it stands, winner and loser of the match is clearly defined. Next step would be to change winner and loser to left_player and right_player while maintaining their according stats. After, iterate through each individual row and randomly flip left_player and right_player so that the outcome column will have 1(left_player winning) or 2(right_player winning).

In [ ]:
stats_df = pd.read_csv("../original_data/Stats/match_stats_2017.csv")
stats_df.head()

In [139]:
stats_files = glob.glob("../original_data/Stats/*.csv")
stats_df = pd.concat((pd.read_csv(f) for f in stats_files))
stats_df.head()

,tourney_order,match_id,match_stats_url_suffix,match_time,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,0,2017-339-r975-n409,/en/scores/2017/339/MS004/match-stats,02:21:00,141.0,23.0,3.0,62.0,97.0,50.0,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
1,0,2017-339-d875-n552,/en/scores/2017/339/MS001/match-stats,01:49:00,109.0,7.0,2.0,52.0,77.0,41.0,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
2,0,2017-339-d875-r975,/en/scores/2017/339/MS002/match-stats,01:28:00,88.0,4.0,1.0,36.0,58.0,27.0,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
3,0,2017-339-n552-w367,/en/scores/2017/339/MS003/match-stats,01:42:00,102.0,1.0,1.0,56.0,77.0,37.0,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
4,0,2017-339-d875-tb69,/en/scores/2017/339/MS005/match-stats,02:05:00,125.0,3.0,3.0,52.0,94.0,42.0,...,19.0,42.0,1.0,14.0,14.0,14.0,29.0,94.0,82.0,176.0


In [140]:
stats_df = stats_df.dropna()

In [141]:
stats_df = stats_df.drop(columns=['loser_total_points_total'])

In [142]:
stats_df = stats_df.rename(columns={"winner_total_points_total": "total_points_played"})

In [143]:
stats_df["outcome"] = np.where(stats_df["winner_total_points_won"] > stats_df["loser_total_points_won"], 1.0, 2.0)

In [146]:
stats_df = stats_df.drop(columns=['tourney_order','match_id','match_stats_url_suffix'
                                  ,'match_time','winner_total_points_won','loser_total_points_won'], axis=1)

In [147]:
stats_df.head()

,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,...,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,outcome
0,141.0,23.0,3.0,62.0,97.0,50.0,62.0,16.0,35.0,6.0,...,62.0,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,1.0
1,109.0,7.0,2.0,52.0,77.0,41.0,52.0,12.0,25.0,5.0,...,52.0,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,1.0
2,88.0,4.0,1.0,36.0,58.0,27.0,36.0,18.0,22.0,0.0,...,36.0,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,1.0
3,102.0,1.0,1.0,56.0,77.0,37.0,56.0,14.0,21.0,4.0,...,56.0,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,1.0
4,125.0,3.0,3.0,52.0,94.0,42.0,52.0,23.0,42.0,13.0,...,52.0,19.0,42.0,1.0,14.0,14.0,14.0,29.0,94.0,1.0


In [9]:
stats_df.dtypes

match_duration                      float64
winner_aces                         float64
winner_double_faults                float64
winner_first_serves_in              float64
winner_first_serves_total           float64
winner_first_serve_points_won       float64
winner_first_serve_points_total     float64
winner_second_serve_points_won      float64
winner_second_serve_points_total    float64
winner_break_points_saved           float64
winner_break_points_serve_total     float64
winner_service_points_won           float64
winner_service_points_total         float64
winner_first_serve_return_won       float64
winner_first_serve_return_total     float64
winner_second_serve_return_won      float64
winner_second_serve_return_total    float64
winner_break_points_converted       float64
winner_break_points_return_total    float64
winner_service_games_played         float64
winner_return_games_played          float64
winner_return_points_won            float64
winner_return_points_total      

In [10]:
X = stats_df.drop(['outcome'], axis=1)
#X = pd.get_dummies(X)
y = stats_df["outcome"]

In [11]:
from sklearn.model_selection import train_test_split
# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state = 1,
                                                    stratify = y)

In [12]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
classifier = LogisticRegression(solver='lbfgs', max_iter=2500, random_state=1)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=2500, random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = classifier.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)
print(acc_score)

0.9996639132437556


In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
class_rep = classification_report_imbalanced(y_test,y_pred)
print(class_rep)

                   pre       rec       spe        f1       geo       iba       sup

        1.0       1.00      1.00      1.00      1.00      1.00      1.00     22311
        2.0       1.00      1.00      1.00      1.00      1.00      1.00      1594

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     23905



In [22]:
# Display the confusion matrix
conf_mat =  confusion_matrix(y_test,y_pred)
print(conf_mat)

[[22310     1]
 [    1  1593]]


In [24]:
# Display the balanced accuracy score, imbalanced classification report, confusion matrix
print(f"The model produced an accuracy score of {acc_score}")
print(" ")
print("Imbalanced classification report")
print(class_rep)
print(" ")
print("Confusion Matrix")
print(" ")
print(conf_mat)

The model produced an accuracy score of 0.9996639132437556
 
Imbalanced classification report
                   pre       rec       spe        f1       geo       iba       sup

        1.0       1.00      1.00      1.00      1.00      1.00      1.00     22311
        2.0       1.00      1.00      1.00      1.00      1.00      1.00      1594

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     23905

 
Confusion Matrix
 
[[22310     1]
 [    1  1593]]


In [106]:
# reading in scores df
scores_files = glob.glob("../original_data/Scores/*.csv")
scores_df = pd.concat((pd.read_csv(f) for f in scores_files))
scores_df.sample(10)

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
71446,2011-500,33,halle,/en/scores/archive/halle/500/2011/results,Round of 16,4,5,Philipp Kohlschreiber,k435,philipp-kohlschreiber,...,7,63 76(5),2,0,13,9,1,0,2011-500-k435-d801,/en/scores/2011/500/MS009/match-stats
42848,2004-495,16,dubai,/en/scores/archive/dubai/495/2004/results,Round of 32,5,9,Gregory Carraz,c400,gregory-carraz,...,WC,60 60,2,0,12,0,0,0,2004-495-c400-a499,/en/scores/2004/495/MS021/match-stats
4788,1992-520,37,roland-garros,/en/scores/archive/roland-garros/520/1992/results,Semi-Finals,2,2,Petr Korda,k148,petr-korda,...,WC,62 76(4) 63,3,0,19,11,1,0,1992-520-k148-l014,/en/scores/1992/520/MS003/match-stats
94856,2016-7581,56,chengdu,/en/scores/archive/chengdu/7581/2016/results,Round of 32,5,6,Marcos Baghdatis,b837,marcos-baghdatis,...,NaN,67(5) 76(5) 76(5),2,1,20,19,2,1,2016-7581-b837-cd85,/en/scores/2016/7581/MS029/match-stats
54871,2007-560,52,us-open,/en/scores/archive/us-open/560/2007/results,1st Round Qualifying,10,13,Paul Capdeville,c634,paul-capdeville,...,NaN,64 64,2,0,12,8,0,0,2007-560-c634-r548,NaN
10355,1993-438,88,moscow,/en/scores/archive/moscow/438/1993/results,Round of 32,5,7,Paul Haarhuis,h223,paul-haarhuis,...,NaN,63 62,2,0,12,5,0,0,1993-438-h223-p273,/en/scores/1993/438/MS019/match-stats
1373,1991-615,35,dusseldorf,/en/scores/archive/dusseldorf/615/1991/results,Round Robin,2,5,Andrei Cherkasov,c260,andrei-cherkasov,...,NaN,64 61,2,0,12,5,0,0,1991-615-c260-e004,NaN
18527,1996-329,29,tokyo,/en/scores/archive/tokyo/329/1996/results,Semi-Finals,2,2,Richey Reneberg,r016,richey-reneberg,...,NaN,36 64 76(4),2,1,16,16,1,0,1996-329-r016-d249,/en/scores/1996/329/MS003/match-stats
83936,2014-403,20,miami,/en/scores/archive/miami/403/2014/results,Round of 128,7,13,Ryan Harrison,h940,ryan-harrison,...,NaN,62 64,2,0,12,6,0,0,2014-403-h940-d874,/en/scores/2014/403/MS078/match-stats
74558,2012-506,13,buenos-aires,/en/scores/archive/buenos-aires/506/2012/results,2nd Round Qualifying,7,4,Igor Andreev,a511,igor-andreev,...,NaN,60 61,2,0,12,1,0,0,2012-506-a511-pd44,/en/scores/2012/506/QS011/match-stats


In [107]:
stats_files = glob.glob("../original_data/Stats/*.csv")
stats_df = pd.concat((pd.read_csv(f) for f in stats_files))
stats_df.head()

,tourney_order,match_id,match_stats_url_suffix,match_time,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,0,2017-339-r975-n409,/en/scores/2017/339/MS004/match-stats,02:21:00,141.0,23.0,3.0,62.0,97.0,50.0,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
1,0,2017-339-d875-n552,/en/scores/2017/339/MS001/match-stats,01:49:00,109.0,7.0,2.0,52.0,77.0,41.0,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
2,0,2017-339-d875-r975,/en/scores/2017/339/MS002/match-stats,01:28:00,88.0,4.0,1.0,36.0,58.0,27.0,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
3,0,2017-339-n552-w367,/en/scores/2017/339/MS003/match-stats,01:42:00,102.0,1.0,1.0,56.0,77.0,37.0,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
4,0,2017-339-d875-tb69,/en/scores/2017/339/MS005/match-stats,02:05:00,125.0,3.0,3.0,52.0,94.0,42.0,...,19.0,42.0,1.0,14.0,14.0,14.0,29.0,94.0,82.0,176.0


In [108]:
stats_scores = pd.merge(scores_df, stats_df, on=["match_id", "match_id"])
stats_scores

,tourney_year_id,tourney_order_x,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Finals,1,1,Grigor Dimitrov,d875,grigor-dimitrov,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
1,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,1,Grigor Dimitrov,d875,grigor-dimitrov,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
2,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,2,Kei Nishikori,n552,kei-nishikori,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
3,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,1,Milos Raonic,r975,milos-raonic,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
4,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,2,Stan Wawrinka,w367,stan-wawrinka,...,24.0,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,8,Dominic Thiem,tb69,dominic-thiem,...,12.0,24.0,3.0,3.0,13.0,13.0,25.0,71.0,74.0,148.0
95769,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,9,Andy Murray,mc10,andy-murray,...,11.0,27.0,1.0,5.0,8.0,9.0,19.0,58.0,43.0,103.0
95770,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,10,Kei Nishikori,n552,kei-nishikori,...,6.0,23.0,0.0,0.0,9.0,8.0,10.0,43.0,41.0,101.0
95771,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,11,Novak Djokovic,d643,novak-djokovic,...,6.0,26.0,0.0,1.0,13.0,13.0,21.0,80.0,75.0,176.0


In [109]:
# Renaming winner columns to player_1 and loser columns to player_2

# Getting all columns names that start with "winner"

winners_cols = []
winners_cols_fixed = []
losers_cols = []
winners_cols_fixed = []

for column in stats_scores:
    
    columnSeriesObj = stats_scores[column]
    if "winner" in column:
        print(column)
        winners_cols.append(column)
        winners_cols_fixed.append(column)
    if "loser" in column:
        print(column)
        losers_cols.append(column)
        losers_cols_fixed.append(column)

winner_name
winner_player_id
winner_slug
loser_name
loser_player_id
loser_slug
winner_seed
loser_seed
winner_sets_won
loser_sets_won
winner_games_won
loser_games_won
winner_tiebreaks_won
loser_tiebreaks_won
winner_aces
winner_double_faults
winner_first_serves_in
winner_first_serves_total
winner_first_serve_points_won
winner_first_serve_points_total
winner_second_serve_points_won
winner_second_serve_points_total
winner_break_points_saved
winner_break_points_serve_total
winner_service_points_won
winner_service_points_total
winner_first_serve_return_won
winner_first_serve_return_total
winner_second_serve_return_won
winner_second_serve_return_total
winner_break_points_converted
winner_break_points_return_total
winner_service_games_played
winner_return_games_played
winner_return_points_won
winner_return_points_total
winner_total_points_won
winner_total_points_total
loser_aces
loser_double_faults
loser_first_serves_in
loser_first_serves_total
loser_first_serve_points_won
loser_first_serve_po

In [110]:
winners_cols

['winner_name',
 'winner_player_id',
 'winner_slug',
 'winner_seed',
 'winner_sets_won',
 'winner_games_won',
 'winner_tiebreaks_won',
 'winner_aces',
 'winner_double_faults',
 'winner_first_serves_in',
 'winner_first_serves_total',
 'winner_first_serve_points_won',
 'winner_first_serve_points_total',
 'winner_second_serve_points_won',
 'winner_second_serve_points_total',
 'winner_break_points_saved',
 'winner_break_points_serve_total',
 'winner_service_points_won',
 'winner_service_points_total',
 'winner_first_serve_return_won',
 'winner_first_serve_return_total',
 'winner_second_serve_return_won',
 'winner_second_serve_return_total',
 'winner_break_points_converted',
 'winner_break_points_return_total',
 'winner_service_games_played',
 'winner_return_games_played',
 'winner_return_points_won',
 'winner_return_points_total',
 'winner_total_points_won',
 'winner_total_points_total']

In [111]:
for i in range(len(winners_cols)):
    winners_cols_fixed[i] = winners_cols_fixed[i].replace("winner", "player_1")

winners_cols_fixed

['player_1_name',
 'player_1_player_id',
 'player_1_slug',
 'player_1_seed',
 'player_1_sets_won',
 'player_1_games_won',
 'player_1_tiebreaks_won',
 'player_1_aces',
 'player_1_double_faults',
 'player_1_first_serves_in',
 'player_1_first_serves_total',
 'player_1_first_serve_points_won',
 'player_1_first_serve_points_total',
 'player_1_second_serve_points_won',
 'player_1_second_serve_points_total',
 'player_1_break_points_saved',
 'player_1_break_points_serve_total',
 'player_1_service_points_won',
 'player_1_service_points_total',
 'player_1_first_serve_return_won',
 'player_1_first_serve_return_total',
 'player_1_second_serve_return_won',
 'player_1_second_serve_return_total',
 'player_1_break_points_converted',
 'player_1_break_points_return_total',
 'player_1_service_games_played',
 'player_1_return_games_played',
 'player_1_return_points_won',
 'player_1_return_points_total',
 'player_1_total_points_won',
 'player_1_total_points_total']

In [112]:
for i in range(len(losers_cols)):
    losers_cols_fixed[i] = losers_cols_fixed[i].replace("loser", "player_2")

losers_cols_fixed

['player_2_name',
 'player_2_player_id',
 'player_2_slug',
 'player_2_seed',
 'player_2_sets_won',
 'player_2_games_won',
 'player_2_tiebreaks_won',
 'player_2_aces',
 'player_2_double_faults',
 'player_2_first_serves_in',
 'player_2_first_serves_total',
 'player_2_first_serve_points_won',
 'player_2_first_serve_points_total',
 'player_2_second_serve_points_won',
 'player_2_second_serve_points_total',
 'player_2_break_points_saved',
 'player_2_break_points_serve_total',
 'player_2_service_points_won',
 'player_2_service_points_total',
 'player_2_first_serve_return_won',
 'player_2_first_serve_return_total',
 'player_2_second_serve_return_won',
 'player_2_second_serve_return_total',
 'player_2_break_points_converted',
 'player_2_break_points_return_total',
 'player_2_service_games_played',
 'player_2_return_games_played',
 'player_2_return_points_won',
 'player_2_return_points_total',
 'player_2_total_points_won',
 'player_2_total_points_total',
 'loser_name',
 'loser_player_id',
 'lose

In [113]:
# Creating dictionary of winner columns
player_1_dict = dict(zip(winners_cols, winners_cols_fixed))
player_2_dict = dict(zip(losers_cols, losers_cols_fixed))

In [114]:
player_1_dict

{'winner_name': 'player_1_name',
 'winner_player_id': 'player_1_player_id',
 'winner_slug': 'player_1_slug',
 'winner_seed': 'player_1_seed',
 'winner_sets_won': 'player_1_sets_won',
 'winner_games_won': 'player_1_games_won',
 'winner_tiebreaks_won': 'player_1_tiebreaks_won',
 'winner_aces': 'player_1_aces',
 'winner_double_faults': 'player_1_double_faults',
 'winner_first_serves_in': 'player_1_first_serves_in',
 'winner_first_serves_total': 'player_1_first_serves_total',
 'winner_first_serve_points_won': 'player_1_first_serve_points_won',
 'winner_first_serve_points_total': 'player_1_first_serve_points_total',
 'winner_second_serve_points_won': 'player_1_second_serve_points_won',
 'winner_second_serve_points_total': 'player_1_second_serve_points_total',
 'winner_break_points_saved': 'player_1_break_points_saved',
 'winner_break_points_serve_total': 'player_1_break_points_serve_total',
 'winner_service_points_won': 'player_1_service_points_won',
 'winner_service_points_total': 'player_

In [115]:
p1_p2_dict = dict(player_1_dict, **player_2_dict)

In [116]:
stats_scores.rename(columns=p1_p2_dict, inplace=True)

In [118]:
# Checking columns after renaming (player_1 is still the winner)
stats_scores.head()

,tourney_year_id,tourney_order_x,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,player_1_name,player_1_player_id,player_1_slug,...,player_2_second_serve_return_won,player_2_second_serve_return_total,player_2_break_points_converted,player_2_break_points_return_total,player_2_service_games_played,player_2_return_games_played,player_2_return_points_won,player_2_return_points_total,player_2_total_points_won,player_2_total_points_total
0,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Finals,1,1,Grigor Dimitrov,d875,grigor-dimitrov,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
1,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,1,Grigor Dimitrov,d875,grigor-dimitrov,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
2,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,2,Kei Nishikori,n552,kei-nishikori,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
3,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,1,Milos Raonic,r975,milos-raonic,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
4,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,2,Stan Wawrinka,w367,stan-wawrinka,...,24.0,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0


In [123]:
stats_scores = stats_scores.drop(columns=['tourney_year_id','tourney_order_x','tourney_slug','tourney_url_suffix'
                                          ,'tourney_round_name','match_order','match_stats_url_suffix_x'
                                          ,'tourney_order_y','match_stats_url_suffix_y'
                                          ,'match_time','match_score_tiebreaks'],axis=1)

In [124]:
stats_scores.head()

,round_order,player_1_name,player_1_player_id,player_1_slug,player_2_name,player_2_player_id,player_2_slug,player_1_seed,player_2_seed,match_score_tiebreaks,...,player_2_second_serve_return_won,player_2_second_serve_return_total,player_2_break_points_converted,player_2_break_points_return_total,player_2_service_games_played,player_2_return_games_played,player_2_return_points_won,player_2_return_points_total,player_2_total_points_won,player_2_total_points_total
0,1,Grigor Dimitrov,d875,grigor-dimitrov,Kei Nishikori,n552,kei-nishikori,7,3,62 26 63,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
1,2,Grigor Dimitrov,d875,grigor-dimitrov,Milos Raonic,r975,milos-raonic,7,1,76(7) 62,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
2,2,Kei Nishikori,n552,kei-nishikori,Stan Wawrinka,w367,stan-wawrinka,3,2,76(3) 63,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
3,3,Milos Raonic,r975,milos-raonic,Rafael Nadal,n409,rafael-nadal,1,5,46 63 64,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
4,3,Stan Wawrinka,w367,stan-wawrinka,Kyle Edmund,e831,kyle-edmund,2,NaN,67(2) 64 64,...,24.0,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0


In [125]:
stats_scores["winner"] = stats_scores["player_1_name"]

In [334]:
# Take all player_1 columns into one df
player_1_df = stats_scores.loc[: , ['player_1' in i for i in stats_scores.columns]]
player_1_df['outcome'] = player_1_df.apply(lambda x: '0', axis=1)
player_1_df

/var/folders/68/x1_csqhx5rlbpjpmwr7hjqw80000gn/T/ipykernel_4104/1931753592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_1_df['outcome'] = player_1_df.apply(lambda x: '0', axis=1)


,player_1_name,player_1_player_id,player_1_seed,player_1_sets_won,player_1_games_won,player_1_tiebreaks_won,player_1_aces,player_1_double_faults,player_1_first_serves_in,player_1_first_serves_total,...,player_1_second_serve_return_total,player_1_break_points_converted,player_1_break_points_return_total,player_1_service_games_played,player_1_return_games_played,player_1_return_points_won,player_1_return_points_total,player_1_total_points_won,player_1_total_points_total,outcome
0,Grigor Dimitrov,d875,7,2,14,0,7.0,2.0,52.0,77.0,...,20.0,3.0,5.0,13.0,12.0,24.0,69.0,77.0,146.0,0
1,Grigor Dimitrov,d875,7,2,13,1,4.0,1.0,36.0,58.0,...,33.0,2.0,4.0,10.0,10.0,21.0,61.0,66.0,119.0,0
2,Kei Nishikori,n552,3,2,13,1,1.0,1.0,56.0,77.0,...,24.0,2.0,2.0,11.0,10.0,24.0,61.0,75.0,138.0,0
3,Milos Raonic,r975,1,2,16,0,23.0,3.0,62.0,97.0,...,23.0,2.0,4.0,15.0,14.0,31.0,84.0,97.0,181.0,0
4,Stan Wawrinka,w367,2,2,18,0,11.0,3.0,67.0,119.0,...,32.0,4.0,10.0,16.0,16.0,34.0,97.0,109.0,216.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,Dominic Thiem,tb69,8,2,13,0,4.0,1.0,47.0,71.0,...,39.0,3.0,8.0,13.0,13.0,28.0,77.0,74.0,148.0,0
95769,Andy Murray,mc10,1,2,12,0,3.0,4.0,31.0,58.0,...,22.0,4.0,5.0,9.0,8.0,21.0,45.0,60.0,103.0,0
95770,Kei Nishikori,n552,5,2,12,0,5.0,1.0,20.0,43.0,...,21.0,4.0,7.0,8.0,9.0,27.0,58.0,60.0,101.0,0
95771,Novak Djokovic,d643,2,2,18,0,3.0,0.0,54.0,80.0,...,32.0,5.0,6.0,13.0,13.0,42.0,96.0,101.0,176.0,0


In [335]:
# Take all player_2 columns into one df
player_2_df = stats_scores.loc[: , ['player_2' in i for i in stats_scores.columns]]
player_2_df['outcome'] = player_2_df.apply(lambda x: '1', axis=1)
player_2_df

/var/folders/68/x1_csqhx5rlbpjpmwr7hjqw80000gn/T/ipykernel_4104/808766751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_2_df['outcome'] = player_2_df.apply(lambda x: '1', axis=1)


,player_2_name,player_2_player_id,player_2_seed,player_2_sets_won,player_2_games_won,player_2_tiebreaks_won,player_2_aces,player_2_double_faults,player_2_first_serves_in,player_2_first_serves_total,...,player_2_second_serve_return_total,player_2_break_points_converted,player_2_break_points_return_total,player_2_service_games_played,player_2_return_games_played,player_2_return_points_won,player_2_return_points_total,player_2_total_points_won,player_2_total_points_total,outcome
0,Kei Nishikori,n552,3,1,11,0,4.0,0.0,49.0,69.0,...,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0,1
1,Milos Raonic,r975,1,0,8,0,4.0,3.0,28.0,61.0,...,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0,1
2,Stan Wawrinka,w367,2,0,9,0,9.0,2.0,37.0,61.0,...,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0,1
3,Rafael Nadal,n409,5,1,13,0,4.0,0.0,61.0,84.0,...,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0,1
4,Kyle Edmund,e831,0,1,15,1,2.0,2.0,65.0,97.0,...,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,Gael Monfils,mc65,6,1,13,0,8.0,5.0,38.0,77.0,...,24.0,3.0,3.0,13.0,13.0,25.0,71.0,74.0,148.0,1
95769,Marin Cilic,c977,7,0,5,0,4.0,1.0,23.0,45.0,...,27.0,1.0,5.0,8.0,9.0,19.0,58.0,43.0,103.0,1
95770,Stan Wawrinka,w367,3,0,5,0,5.0,3.0,37.0,58.0,...,23.0,0.0,0.0,9.0,8.0,10.0,43.0,41.0,101.0,1
95771,Dominic Thiem,tb69,8,1,9,1,4.0,3.0,64.0,96.0,...,26.0,0.0,1.0,13.0,13.0,21.0,80.0,75.0,176.0,1


In [336]:
player_2_df.columns

Index(['player_2_name', 'player_2_player_id', 'player_2_seed',
       'player_2_sets_won', 'player_2_games_won', 'player_2_tiebreaks_won',
       'player_2_aces', 'player_2_double_faults', 'player_2_first_serves_in',
       'player_2_first_serves_total', 'player_2_first_serve_points_won',
       'player_2_first_serve_points_total', 'player_2_second_serve_points_won',
       'player_2_second_serve_points_total', 'player_2_break_points_saved',
       'player_2_break_points_serve_total', 'player_2_service_points_won',
       'player_2_service_points_total', 'player_2_first_serve_return_won',
       'player_2_first_serve_return_total', 'player_2_second_serve_return_won',
       'player_2_second_serve_return_total', 'player_2_break_points_converted',
       'player_2_break_points_return_total', 'player_2_service_games_played',
       'player_2_return_games_played', 'player_2_return_points_won',
       'player_2_return_points_total', 'player_2_total_points_won',
       'player_2_total_points

In [337]:
play_2_cols = []
play_2_fixed = []
for column in player_2_df:
    columnSeriesObj = player_2_df[column]
    if "player_2" in column:
        print(column)
        play_2_cols.append(column)
        play_2_fixed.append(column)

player_2_name
player_2_player_id
player_2_seed
player_2_sets_won
player_2_games_won
player_2_tiebreaks_won
player_2_aces
player_2_double_faults
player_2_first_serves_in
player_2_first_serves_total
player_2_first_serve_points_won
player_2_first_serve_points_total
player_2_second_serve_points_won
player_2_second_serve_points_total
player_2_break_points_saved
player_2_break_points_serve_total
player_2_service_points_won
player_2_service_points_total
player_2_first_serve_return_won
player_2_first_serve_return_total
player_2_second_serve_return_won
player_2_second_serve_return_total
player_2_break_points_converted
player_2_break_points_return_total
player_2_service_games_played
player_2_return_games_played
player_2_return_points_won
player_2_return_points_total
player_2_total_points_won
player_2_total_points_total


In [338]:
# Renaming all player_2 columns to player_1
for i in range(len(play_2_cols)):
    play_2_fixed[i] = play_2_fixed[i].replace("player_2", "player_1")

play_2_fixed

['player_1_name',
 'player_1_player_id',
 'player_1_seed',
 'player_1_sets_won',
 'player_1_games_won',
 'player_1_tiebreaks_won',
 'player_1_aces',
 'player_1_double_faults',
 'player_1_first_serves_in',
 'player_1_first_serves_total',
 'player_1_first_serve_points_won',
 'player_1_first_serve_points_total',
 'player_1_second_serve_points_won',
 'player_1_second_serve_points_total',
 'player_1_break_points_saved',
 'player_1_break_points_serve_total',
 'player_1_service_points_won',
 'player_1_service_points_total',
 'player_1_first_serve_return_won',
 'player_1_first_serve_return_total',
 'player_1_second_serve_return_won',
 'player_1_second_serve_return_total',
 'player_1_break_points_converted',
 'player_1_break_points_return_total',
 'player_1_service_games_played',
 'player_1_return_games_played',
 'player_1_return_points_won',
 'player_1_return_points_total',
 'player_1_total_points_won',
 'player_1_total_points_total']

In [339]:
play_dict = dict(zip(play_2_cols, play_2_fixed))
player_2_df.rename(columns = play_dict, inplace=True)

/var/folders/68/x1_csqhx5rlbpjpmwr7hjqw80000gn/T/ipykernel_4104/1269003972.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_2_df.rename(columns = play_dict, inplace=True)


In [340]:
player_1_df

,player_1_name,player_1_player_id,player_1_seed,player_1_sets_won,player_1_games_won,player_1_tiebreaks_won,player_1_aces,player_1_double_faults,player_1_first_serves_in,player_1_first_serves_total,...,player_1_second_serve_return_total,player_1_break_points_converted,player_1_break_points_return_total,player_1_service_games_played,player_1_return_games_played,player_1_return_points_won,player_1_return_points_total,player_1_total_points_won,player_1_total_points_total,outcome
0,Grigor Dimitrov,d875,7,2,14,0,7.0,2.0,52.0,77.0,...,20.0,3.0,5.0,13.0,12.0,24.0,69.0,77.0,146.0,0
1,Grigor Dimitrov,d875,7,2,13,1,4.0,1.0,36.0,58.0,...,33.0,2.0,4.0,10.0,10.0,21.0,61.0,66.0,119.0,0
2,Kei Nishikori,n552,3,2,13,1,1.0,1.0,56.0,77.0,...,24.0,2.0,2.0,11.0,10.0,24.0,61.0,75.0,138.0,0
3,Milos Raonic,r975,1,2,16,0,23.0,3.0,62.0,97.0,...,23.0,2.0,4.0,15.0,14.0,31.0,84.0,97.0,181.0,0
4,Stan Wawrinka,w367,2,2,18,0,11.0,3.0,67.0,119.0,...,32.0,4.0,10.0,16.0,16.0,34.0,97.0,109.0,216.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,Dominic Thiem,tb69,8,2,13,0,4.0,1.0,47.0,71.0,...,39.0,3.0,8.0,13.0,13.0,28.0,77.0,74.0,148.0,0
95769,Andy Murray,mc10,1,2,12,0,3.0,4.0,31.0,58.0,...,22.0,4.0,5.0,9.0,8.0,21.0,45.0,60.0,103.0,0
95770,Kei Nishikori,n552,5,2,12,0,5.0,1.0,20.0,43.0,...,21.0,4.0,7.0,8.0,9.0,27.0,58.0,60.0,101.0,0
95771,Novak Djokovic,d643,2,2,18,0,3.0,0.0,54.0,80.0,...,32.0,5.0,6.0,13.0,13.0,42.0,96.0,101.0,176.0,0


In [341]:
player_2_df

,player_1_name,player_1_player_id,player_1_seed,player_1_sets_won,player_1_games_won,player_1_tiebreaks_won,player_1_aces,player_1_double_faults,player_1_first_serves_in,player_1_first_serves_total,...,player_1_second_serve_return_total,player_1_break_points_converted,player_1_break_points_return_total,player_1_service_games_played,player_1_return_games_played,player_1_return_points_won,player_1_return_points_total,player_1_total_points_won,player_1_total_points_total,outcome
0,Kei Nishikori,n552,3,1,11,0,4.0,0.0,49.0,69.0,...,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0,1
1,Milos Raonic,r975,1,0,8,0,4.0,3.0,28.0,61.0,...,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0,1
2,Stan Wawrinka,w367,2,0,9,0,9.0,2.0,37.0,61.0,...,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0,1
3,Rafael Nadal,n409,5,1,13,0,4.0,0.0,61.0,84.0,...,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0,1
4,Kyle Edmund,e831,0,1,15,1,2.0,2.0,65.0,97.0,...,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,Gael Monfils,mc65,6,1,13,0,8.0,5.0,38.0,77.0,...,24.0,3.0,3.0,13.0,13.0,25.0,71.0,74.0,148.0,1
95769,Marin Cilic,c977,7,0,5,0,4.0,1.0,23.0,45.0,...,27.0,1.0,5.0,8.0,9.0,19.0,58.0,43.0,103.0,1
95770,Stan Wawrinka,w367,3,0,5,0,5.0,3.0,37.0,58.0,...,23.0,0.0,0.0,9.0,8.0,10.0,43.0,41.0,101.0,1
95771,Dominic Thiem,tb69,8,1,9,1,4.0,3.0,64.0,96.0,...,26.0,0.0,1.0,13.0,13.0,21.0,80.0,75.0,176.0,1


In [342]:
# putting player_1_df and player_2_df back together
all_player_matches = pd.concat([player_1_df,player_2_df])
all_player_matches

,player_1_name,player_1_player_id,player_1_seed,player_1_sets_won,player_1_games_won,player_1_tiebreaks_won,player_1_aces,player_1_double_faults,player_1_first_serves_in,player_1_first_serves_total,...,player_1_second_serve_return_total,player_1_break_points_converted,player_1_break_points_return_total,player_1_service_games_played,player_1_return_games_played,player_1_return_points_won,player_1_return_points_total,player_1_total_points_won,player_1_total_points_total,outcome
0,Grigor Dimitrov,d875,7,2,14,0,7.0,2.0,52.0,77.0,...,20.0,3.0,5.0,13.0,12.0,24.0,69.0,77.0,146.0,0
1,Grigor Dimitrov,d875,7,2,13,1,4.0,1.0,36.0,58.0,...,33.0,2.0,4.0,10.0,10.0,21.0,61.0,66.0,119.0,0
2,Kei Nishikori,n552,3,2,13,1,1.0,1.0,56.0,77.0,...,24.0,2.0,2.0,11.0,10.0,24.0,61.0,75.0,138.0,0
3,Milos Raonic,r975,1,2,16,0,23.0,3.0,62.0,97.0,...,23.0,2.0,4.0,15.0,14.0,31.0,84.0,97.0,181.0,0
4,Stan Wawrinka,w367,2,2,18,0,11.0,3.0,67.0,119.0,...,32.0,4.0,10.0,16.0,16.0,34.0,97.0,109.0,216.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,Gael Monfils,mc65,6,1,13,0,8.0,5.0,38.0,77.0,...,24.0,3.0,3.0,13.0,13.0,25.0,71.0,74.0,148.0,1
95769,Marin Cilic,c977,7,0,5,0,4.0,1.0,23.0,45.0,...,27.0,1.0,5.0,8.0,9.0,19.0,58.0,43.0,103.0,1
95770,Stan Wawrinka,w367,3,0,5,0,5.0,3.0,37.0,58.0,...,23.0,0.0,0.0,9.0,8.0,10.0,43.0,41.0,101.0,1
95771,Dominic Thiem,tb69,8,1,9,1,4.0,3.0,64.0,96.0,...,26.0,0.0,1.0,13.0,13.0,21.0,80.0,75.0,176.0,1


In [343]:
all_player_matches.drop('player_1_name',axis=1,inplace=True)

In [344]:
all_player_matches

,player_1_player_id,player_1_seed,player_1_sets_won,player_1_games_won,player_1_tiebreaks_won,player_1_aces,player_1_double_faults,player_1_first_serves_in,player_1_first_serves_total,player_1_first_serve_points_won,...,player_1_second_serve_return_total,player_1_break_points_converted,player_1_break_points_return_total,player_1_service_games_played,player_1_return_games_played,player_1_return_points_won,player_1_return_points_total,player_1_total_points_won,player_1_total_points_total,outcome
0,d875,7,2,14,0,7.0,2.0,52.0,77.0,41.0,...,20.0,3.0,5.0,13.0,12.0,24.0,69.0,77.0,146.0,0
1,d875,7,2,13,1,4.0,1.0,36.0,58.0,27.0,...,33.0,2.0,4.0,10.0,10.0,21.0,61.0,66.0,119.0,0
2,n552,3,2,13,1,1.0,1.0,56.0,77.0,37.0,...,24.0,2.0,2.0,11.0,10.0,24.0,61.0,75.0,138.0,0
3,r975,1,2,16,0,23.0,3.0,62.0,97.0,50.0,...,23.0,2.0,4.0,15.0,14.0,31.0,84.0,97.0,181.0,0
4,w367,2,2,18,0,11.0,3.0,67.0,119.0,47.0,...,32.0,4.0,10.0,16.0,16.0,34.0,97.0,109.0,216.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,mc65,6,1,13,0,8.0,5.0,38.0,77.0,31.0,...,24.0,3.0,3.0,13.0,13.0,25.0,71.0,74.0,148.0,1
95769,c977,7,0,5,0,4.0,1.0,23.0,45.0,15.0,...,27.0,1.0,5.0,8.0,9.0,19.0,58.0,43.0,103.0,1
95770,w367,3,0,5,0,5.0,3.0,37.0,58.0,23.0,...,23.0,0.0,0.0,9.0,8.0,10.0,43.0,41.0,101.0,1
95771,tb69,8,1,9,1,4.0,3.0,64.0,96.0,43.0,...,26.0,0.0,1.0,13.0,13.0,21.0,80.0,75.0,176.0,1


In [346]:
# Encode player id
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

all_player_matches['player_id_encoded'] = le.fit_transform(all_player_matches['player_1_player_id'])

In [348]:
all_player_matches.drop('player_1_player_id',axis=1,inplace=True)

In [355]:
all_player_matches.drop('player_1_seed',axis=1,inplace=True)

In [364]:
all_player_matches[all_player_matches.isnull().any(axis=1)]

,player_1_sets_won,player_1_games_won,player_1_tiebreaks_won,player_1_aces,player_1_double_faults,player_1_first_serves_in,player_1_first_serves_total,player_1_first_serve_points_won,player_1_first_serve_points_total,player_1_second_serve_points_won,...,player_1_break_points_converted,player_1_break_points_return_total,player_1_service_games_played,player_1_return_games_played,player_1_return_points_won,player_1_return_points_total,player_1_total_points_won,player_1_total_points_total,outcome,player_id_encoded
4944,2,13,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3416
4945,2,16,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7
4946,2,13,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3416
4947,2,13,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3416
4948,2,18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55661,0,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,606
55675,1,12,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1098
56200,0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3372
67266,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2230


In [367]:
all_player_matches = all_player_matches.dropna()

In [386]:
all_player_matches

,player_1_sets_won,player_1_games_won,player_1_tiebreaks_won,player_1_aces,player_1_double_faults,player_1_first_serves_in,player_1_first_serves_total,player_1_first_serve_points_won,player_1_first_serve_points_total,player_1_second_serve_points_won,...,player_1_break_points_converted,player_1_break_points_return_total,player_1_service_games_played,player_1_return_games_played,player_1_return_points_won,player_1_return_points_total,player_1_total_points_won,player_1_total_points_total,outcome,player_id_encoded
0,2,14,0,7.0,2.0,52.0,77.0,41.0,52.0,12.0,...,3.0,5.0,13.0,12.0,24.0,69.0,77.0,146.0,0,772
1,2,13,1,4.0,1.0,36.0,58.0,27.0,36.0,18.0,...,2.0,4.0,10.0,10.0,21.0,61.0,66.0,119.0,0,772
2,2,13,1,1.0,1.0,56.0,77.0,37.0,56.0,14.0,...,2.0,2.0,11.0,10.0,24.0,61.0,75.0,138.0,0,2221
3,2,16,0,23.0,3.0,62.0,97.0,50.0,62.0,16.0,...,2.0,4.0,15.0,14.0,31.0,84.0,97.0,181.0,0,2636
4,2,18,0,11.0,3.0,67.0,119.0,47.0,67.0,28.0,...,4.0,10.0,16.0,16.0,34.0,97.0,109.0,216.0,0,3351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,1,13,0,8.0,5.0,38.0,77.0,31.0,38.0,18.0,...,3.0,3.0,13.0,13.0,25.0,71.0,74.0,148.0,1,2010
95769,0,5,0,4.0,1.0,23.0,45.0,15.0,23.0,9.0,...,1.0,5.0,8.0,9.0,19.0,58.0,43.0,103.0,1,566
95770,0,5,0,5.0,3.0,37.0,58.0,23.0,37.0,8.0,...,0.0,0.0,9.0,8.0,10.0,43.0,41.0,101.0,1,3351
95771,1,9,1,4.0,3.0,64.0,96.0,43.0,64.0,11.0,...,0.0,1.0,13.0,13.0,21.0,80.0,75.0,176.0,1,3146


# MACHINE LEARNING MODEL ATTEMPT #2
# TAKE A LOOK AT THIS DATA LEAKAGE?

In [378]:
X = all_player_matches.drop(['outcome'],axis=1)
y = all_player_matches['outcome']

In [379]:
from sklearn.model_selection import train_test_split
# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state = 1,
                                                    test_size=0.2)

In [380]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [381]:
classifier = LogisticRegression(solver='lbfgs', max_iter=5000, random_state=1)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=5000, random_state=1)

In [382]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = classifier.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)
print(acc_score)

0.9963926647602932


In [383]:
# Display the confusion matrix
conf_mat =  confusion_matrix(y_test,y_pred)
print(conf_mat)

[[19172    71]
 [   67 18940]]


In [384]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
class_rep = classification_report_imbalanced(y_test,y_pred)
print(class_rep)

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      0.99     19243
          1       1.00      1.00      1.00      1.00      1.00      0.99     19007

avg / total       1.00      1.00      1.00      1.00      1.00      0.99     38250



In [311]:
stats_scores.columns

Index(['round_order', 'player_1_name', 'player_1_player_id', 'player_2_name',
       'player_2_player_id', 'player_1_seed', 'player_2_seed',
       'player_1_sets_won', 'player_2_sets_won', 'player_1_games_won',
       'player_2_games_won', 'player_1_tiebreaks_won',
       'player_2_tiebreaks_won', 'match_id', 'match_duration', 'player_1_aces',
       'player_1_double_faults', 'player_1_first_serves_in',
       'player_1_first_serves_total', 'player_1_first_serve_points_won',
       'player_1_first_serve_points_total', 'player_1_second_serve_points_won',
       'player_1_second_serve_points_total', 'player_1_break_points_saved',
       'player_1_break_points_serve_total', 'player_1_service_points_won',
       'player_1_service_points_total', 'player_1_first_serve_return_won',
       'player_1_first_serve_return_total', 'player_1_second_serve_return_won',
       'player_1_second_serve_return_total', 'player_1_break_points_converted',
       'player_1_break_points_return_total', 'player_

In [128]:
stats_scores.head()

,round_order,player_1_name,player_1_player_id,player_1_slug,player_2_name,player_2_player_id,player_2_slug,player_1_seed,player_2_seed,match_score_tiebreaks,...,player_2_second_serve_return_total,player_2_break_points_converted,player_2_break_points_return_total,player_2_service_games_played,player_2_return_games_played,player_2_return_points_won,player_2_return_points_total,player_2_total_points_won,player_2_total_points_total,winner
0,1,Grigor Dimitrov,d875,grigor-dimitrov,Kei Nishikori,n552,kei-nishikori,7,3,62 26 63,...,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0,Grigor Dimitrov
1,2,Grigor Dimitrov,d875,grigor-dimitrov,Milos Raonic,r975,milos-raonic,7,1,76(7) 62,...,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0,Grigor Dimitrov
2,2,Kei Nishikori,n552,kei-nishikori,Stan Wawrinka,w367,stan-wawrinka,3,2,76(3) 63,...,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0,Kei Nishikori
3,3,Milos Raonic,r975,milos-raonic,Rafael Nadal,n409,rafael-nadal,1,5,46 63 64,...,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0,Milos Raonic
4,3,Stan Wawrinka,w367,stan-wawrinka,Kyle Edmund,e831,kyle-edmund,2,NaN,67(2) 64 64,...,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0,Stan Wawrinka


In [130]:
# Filling seeded columns with 0 for NaN

stats_scores["player_1_seed"] = stats_scores['player_1_seed'].fillna(0)
stats_scores['player_2_seed'] = stats_scores['player_2_seed'].fillna(0)

In [137]:
stats_scores = stats_scores.drop(columns=['player_1_slug','player_2_slug'],axis=1)

In [138]:
stats_scores.head()

,round_order,player_1_name,player_1_player_id,player_2_name,player_2_player_id,player_1_seed,player_2_seed,player_1_sets_won,player_2_sets_won,player_1_games_won,...,player_2_second_serve_return_total,player_2_break_points_converted,player_2_break_points_return_total,player_2_service_games_played,player_2_return_games_played,player_2_return_points_won,player_2_return_points_total,player_2_total_points_won,player_2_total_points_total,winner
0,1,Grigor Dimitrov,d875,Kei Nishikori,n552,7,3,2,1,14,...,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0,Grigor Dimitrov
1,2,Grigor Dimitrov,d875,Milos Raonic,r975,7,1,2,0,13,...,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0,Grigor Dimitrov
2,2,Kei Nishikori,n552,Stan Wawrinka,w367,3,2,2,0,13,...,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0,Kei Nishikori
3,3,Milos Raonic,r975,Rafael Nadal,n409,1,5,2,1,16,...,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0,Milos Raonic
4,3,Stan Wawrinka,w367,Kyle Edmund,e831,2,0,2,1,18,...,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0,Stan Wawrinka


In [169]:
stats_scores[['player_2_seed', 'player_1_seed']].dtypes

player_2_seed    object
player_1_seed    object
dtype: object

In [156]:
# Encode players by choice of name and encode the player seeds
# Outcome will be player_1 wins or

pen = stats_scores[['player_1_name', 'player_1_player_id']].drop_duplicates()
print(pen.shape)

(2028, 2)


In [164]:
stats_scores['player_2_seed'].unique()

array(['3', '1', '2', '5', 0, 'WC', '4', 'Q', '6', '8', '7', 'PR', 'LL',
       '7 WC', 'AL', '9', '15', '12', '11', '13', '31', '25', '29', '32',
       '24', '10', '20', '23', '21', '18', '27', '30', '14', '19', '16',
       '22', '26', '28', '17', '5 WC', 'S', '4 WC', '1 WC', '6 WC',
       '8 WC', '3 WC', '2 WC', '14 WC', '33', '16 WC', '13 Q', '10 WC',
       'SE', '11 WC', '9 WC', '13 WC', '12 WC', '5 Q', '34', '7 LL', '35',
       '30 WC', '15 WC', '3 AL', '8 AL', '9 AL', '2 AL', '7 AL'],
      dtype=object)

In [304]:
stats_scores.sample(5)

,round_order,player_1_name,player_1_player_id,player_2_name,player_2_player_id,player_1_seed,player_2_seed,player_1_sets_won,player_2_sets_won,player_1_games_won,...,player_2_second_serve_return_total,player_2_break_points_converted,player_2_break_points_return_total,player_2_service_games_played,player_2_return_games_played,player_2_return_points_won,player_2_return_points_total,player_2_total_points_won,player_2_total_points_total,winner
75557,7,Guillermo Garcia-Lopez,g476,Rajeev Ram,r548,Q,Q,2,0,13,...,26.0,1.0,5.0,11.0,11.0,20.0,69.0,70.0,150.0,Guillermo Garcia-Lopez
34650,6,Sjeng Schalken,s572,Christophe Rochus,r336,0,0,3,0,18,...,39.0,2.0,5.0,13.0,13.0,28.0,81.0,80.0,187.0,Sjeng Schalken
63171,9,Sebastien De Chaunac,d351,Pablo Santos,sb15,0,0,2,1,18,...,50.0,1.0,11.0,14.0,14.0,29.0,90.0,78.0,176.0,Sebastien De Chaunac
33794,6,Andre Agassi,a092,Andy Roddick,r485,1,WC,2,0,12,...,16.0,0.0,6.0,9.0,8.0,12.0,48.0,43.0,106.0,Andre Agassi
61523,5,Izak Van der Merwe,v379,Benjamin Becker,b896,WC,0,2,0,14,...,30.0,2.0,2.0,12.0,12.0,19.0,67.0,66.0,144.0,Izak Van der Merwe


In [289]:
# Reading in player_overview
file_path = "../original_data/player_rankings/player_overviews.csv"

players_df = pd.read_csv(file_path)
players_df.head()

,player_id,player_slug,first_name,last_name,player_url,flag_code,residence,birthplace,birthdate,birth_year,birth_month,birth_day,turned_pro,weight_lbs,weight_kg,height_ft,height_inches,height_cm,handedness,backhand
0,a002,ricardo-acuna,Ricardo,Acuna,http://www.atpworldtour.com/en/players/ricardo...,CHI,"Jupiter, FL, USA","Santiago, Chile",1958.01.13,1958.0,1.0,13.0,0.0,150.0,68.0,"5'9""",69.0,175.0,NaN,NaN
1,a001,sadiq-abdullahi,Sadiq,Abdullahi,http://www.atpworldtour.com/en/players/sadiq-a...,NGR,NaN,NaN,1960.02.02,1960.0,2.0,2.0,0.0,0.0,0.0,"0'0""",0.0,0.0,NaN,NaN
2,a005,nelson-aerts,Nelson,Aerts,http://www.atpworldtour.com/en/players/nelson-...,BRA,NaN,"Cachoeira Do Sul, Brazil",1963.04.25,1963.0,4.0,25.0,0.0,165.0,75.0,"6'2""",74.0,188.0,NaN,NaN
3,a004,egan-adams,Egan,Adams,http://www.atpworldtour.com/en/players/egan-ad...,USA,"Palmetto, FL, USA","Miami Beach, FL, USA",1959.06.15,1959.0,6.0,15.0,0.0,160.0,73.0,"5'10""",70.0,178.0,NaN,NaN
4,a006,ronald-agenor,Ronald,Agenor,http://www.atpworldtour.com/en/players/ronald-...,USA,"Beverly Hills, California, USA","Rabat, Morocco",1964.11.13,1964.0,11.0,13.0,1983.0,180.0,82.0,"5'11""",71.0,180.0,NaN,NaN


In [250]:
# removing inactive players and keeping recent active players
players_df = players_df[players_df['handedness'].notna()]
print(players_df['handedness'].unique())
print(players_df['backhand'].unique())

['Right-Handed' 'Left-Handed' 'Ambidextrous']
['One-Handed Backhand' 'Two-Handed Backhand']


In [226]:
# Encoding player names
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

test = players_df.copy()
test['player_encoded'] = le.fit_transform(players_df['player_id'])
test[['player_encoded', 'player_slug', 'player_id']]

,player_encoded,player_slug,player_id
41,0,jorge-andrew,a061
42,1,arthur-ashe,a063
53,2,andre-agassi,a092
105,3,jan-frode-andersen,a276
113,4,hugo-armando,a311
...,...,...,...
10881,1131,zhizhen-zhang,z371
10883,1132,barlaham-zuluaga-gaviria,z381
10886,1133,fran-zvonimir-zgombic,z395
10892,1134,zhicheng-zhu,z412


In [263]:
# df of player_encoded, player_slug, player_id
players_enc = test[['player_encoded', 'player_slug', 'player_id']]
players_enc

,player_encoded,player_slug,player_id
41,0,jorge-andrew,a061
42,1,arthur-ashe,a063
53,2,andre-agassi,a092
105,3,jan-frode-andersen,a276
113,4,hugo-armando,a311
...,...,...,...
10881,1131,zhizhen-zhang,z371
10883,1132,barlaham-zuluaga-gaviria,z381
10886,1133,fran-zvonimir-zgombic,z395
10892,1134,zhicheng-zhu,z412


In [227]:
# Encode handedness and backhand
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
backhand_dict = {
    'One-Handed Backhand': 0,
    'Two-Handed Backhand' : 1
}
test['backhand'] = le.fit_transform(test['backhand'])
# handedness dict
handedness_dict = {
    "Right-Handed" : 0,
    "Left-Handed" : 1,
    "Ambidextrous" : 2
}

test['handedness'] = test['handedness'].apply(lambda x: handedness_dict[x])

In [228]:
#test['backhand'].nunique(dropna=False)
print(test['handedness'].unique())

[0 1 2]


In [237]:
test.head()

,player_id,player_slug,first_name,last_name,player_url,flag_code,residence,birthplace,birthdate,birth_year,...,birth_day,turned_pro,weight_lbs,weight_kg,height_ft,height_inches,height_cm,handedness,backhand,player_encoded
41,a061,jorge-andrew,Jorge,Andrew,http://www.atpworldtour.com/en/players/jorge-a...,VEN,"Lexington, SC, U.S.A.","Caracas, Venezuela",1951.11.02,1951.0,...,2.0,NaN,195.0,88.0,"5'11""",71.0,180.0,0,0,0
42,a063,arthur-ashe,Arthur,Ashe,http://www.atpworldtour.com/en/players/arthur-...,USA,NaN,"Richmond, VA",1943.07.10,1943.0,...,10.0,1969.0,160.0,73.0,"6'1""",73.0,185.0,0,0,1
53,a092,andre-agassi,Andre,Agassi,http://www.atpworldtour.com/en/players/andre-a...,USA,"Las Vegas, Nevada, USA","Las Vegas, Nevada, USA",1970.04.29,1970.0,...,29.0,1986.0,177.0,80.0,"5'11""",71.0,180.0,0,1,2
105,a276,jan-frode-andersen,Jan Frode,Andersen,http://www.atpworldtour.com/en/players/jan-fro...,NOR,"Oslo, Norway","Asker, Norway",1972.08.29,1972.0,...,29.0,1997.0,187.0,85.0,"6'3""",75.0,191.0,1,1,3
113,a311,hugo-armando,Hugo,Armando,http://www.atpworldtour.com/en/players/hugo-ar...,USA,"Bradenton, Florida, USA","Miami, Florida, USA",1978.05.27,1978.0,...,27.0,1997.0,180.0,82.0,"6'0""",72.0,183.0,0,0,4


In [239]:
# merge player_overviews with stats_score on player_id
test_2 = test.merge(stats_scores, left_on="player_id", right_on="player_1_player_id")

In [265]:
test_2[['player_id','player_encoded']]

,player_id,player_encoded
0,a092,2
1,a092,2
2,a092,2
3,a092,2
4,a092,2
...,...,...
66737,z371,1131
66738,z371,1131
66739,z371,1131
66740,z371,1131


In [254]:
test_2 = test_2.drop(columns=['first_name','last_name','player_url','height_ft','height_inches','birthdate',
                            'birth_month','birth_day','weight_lbs'], axis=1)

In [257]:
test_2 = test_2.drop(columns=['residence','birthplace'],axis=1)

In [259]:
test_2 = test_2.drop(columns=['player_1_total_points_total'])
test_2 = test_2.rename(columns={"player_2_total_points_total": "total_points_played"})